In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from sklearn.ensemble import IsolationForest
from datetime import datetime
import calendar
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Configurações para visualização
plt.style.use('ggplot')
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

def load_data():
    """
    Carrega os dados do banco SQLite e retorna os DataFrames necessários
    """
    print("Carregando dados do banco SQLite...")
    
    # Conexão com o banco de dados
    db_path = r'C:\Users\louis\datatech\Database\walmart_fraudes.db'
    conn = sqlite3.connect(db_path)
    
    # Carregar tabelas como DataFrames
    orders_df = pd.read_sql("SELECT * FROM orders", conn)
    drivers_df = pd.read_sql("SELECT * FROM drivers", conn)
    customers_df = pd.read_sql("SELECT * FROM customers", conn)
    missing_items_df = pd.read_sql("SELECT * FROM missing_items", conn)
    products_df = pd.read_sql("SELECT * FROM products", conn)
    
    # Processamento inicial dos dados
    orders_df['date'] = pd.to_datetime(orders_df['date'])
    orders_df['month'] = orders_df['date'].dt.month
    orders_df['day_of_week'] = orders_df['date'].dt.dayofweek
    orders_df['day_name'] = orders_df['date'].dt.day_name()
    
    orders_df['total_items'] = orders_df['items_delivered'] + orders_df['items_missing']
    orders_df['missing_ratio'] = orders_df['items_missing'] / orders_df['total_items']
    
    print(f"Dados carregados com sucesso! Total de {len(orders_df)} pedidos.")
    
    return conn, orders_df, drivers_df, customers_df, missing_items_df, products_df

def exploratory_data_analysis(conn, orders_df, drivers_df, customers_df, missing_items_df, products_df):
    """
    Realiza análise exploratória dos dados
    """
    print("\n=== INICIANDO ANÁLISE EXPLORATÓRIA DE DADOS ===\n")
    
    results = {}
    
    # ---------- 1. ANÁLISE UNIVARIADA ----------
    print("1. Análise Univariada")
    
    # Estatísticas descritivas básicas
    print("\nEstatísticas descritivas dos pedidos:")
    order_stats = orders_df.describe(include='all')
    print(order_stats[['order_amount', 'items_delivered', 'items_missing', 'total_items', 'missing_ratio']])
    
    # Distribuição de valores dos pedidos
    plt.figure(figsize=(10, 6))
    sns.histplot(orders_df['order_amount'], kde=True, bins=30)
    plt.title('Distribuição do Valor dos Pedidos')
    plt.xlabel('Valor do Pedido ($)')
    plt.ylabel('Frequência')
    plt.savefig('figuras/distribuicao_valor_pedidos.png')
    
    # Distribuição de itens perdidos
    plt.figure(figsize=(10, 6))
    sns.countplot(x='items_missing', data=orders_df)
    plt.title('Distribuição de Itens Perdidos por Pedido')
    plt.xlabel('Número de Itens Perdidos')
    plt.ylabel('Contagem de Pedidos')
    plt.savefig('figuras/distribuicao_itens_perdidos.png')
    
    # Distribuição de idade dos motoristas
    plt.figure(figsize=(10, 6))
    sns.histplot(drivers_df['age'], kde=True, bins=20)
    plt.title('Distribuição de Idade dos Motoristas')
    plt.xlabel('Idade')
    plt.ylabel('Frequência')
    plt.savefig('figuras/distribuicao_idade_motoristas.png')
    
    # Distribuição de idade dos clientes
    plt.figure(figsize=(10, 6))
    sns.histplot(customers_df['customer_age'], kde=True, bins=20)
    plt.title('Distribuição de Idade dos Clientes')
    plt.xlabel('Idade')
    plt.ylabel('Frequência')
    plt.savefig('figuras/distribuicao_idade_clientes.png')
    
    # ---------- 2. ANÁLISE BIVARIADA ----------
    print("\n2. Análise Bivariada")
    
    # Relação entre valor do pedido e itens perdidos
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='items_missing', y='order_amount', data=orders_df)
    plt.title('Relação entre Valor do Pedido e Itens Perdidos')
    plt.xlabel('Número de Itens Perdidos')
    plt.ylabel('Valor do Pedido ($)')
    plt.savefig('figuras/relacao_valor_itens_perdidos.png')
    
    # Relação entre horário de entrega e itens perdidos
    orders_df['delivery_hour_only'] = orders_df['delivery_hour'].apply(lambda x: int(x.split(':')[0]))
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='delivery_hour_only', y='items_missing', data=orders_df)
    plt.title('Relação entre Horário de Entrega e Itens Perdidos')
    plt.xlabel('Hora de Entrega (hora do dia)')
    plt.ylabel('Número de Itens Perdidos')
    plt.xticks(rotation=90)
    plt.savefig('figuras/relacao_hora_itens_perdidos.png')
    
    # Correlação entre variáveis numéricas
    numeric_cols = ['order_amount', 'items_delivered', 'items_missing', 'total_items', 'missing_ratio', 'delivery_hour_only']
    corr_matrix = orders_df[numeric_cols].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Matriz de Correlação entre Variáveis')
    plt.savefig('figuras/matriz_correlacao.png')
    
    # ---------- 3. ANÁLISE GEOGRÁFICA ----------
    print("\n3. Análise Geográfica")
    
    # Número de pedidos por região
    region_orders = orders_df.groupby('region').size().reset_index(name='order_count')
    region_orders = region_orders.sort_values('order_count', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='region', y='order_count', data=region_orders)
    plt.title('Número de Pedidos por Região')
    plt.xlabel('Região')
    plt.ylabel('Número de Pedidos')
    plt.xticks(rotation=45)
    plt.savefig('figuras/pedidos_por_regiao.png')
    
    # Média de itens perdidos por região
    region_missing = orders_df.groupby('region')['items_missing'].mean().reset_index(name='avg_missing_items')
    region_missing = region_missing.sort_values('avg_missing_items', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='region', y='avg_missing_items', data=region_missing)
    plt.title('Média de Itens Perdidos por Região')
    plt.xlabel('Região')
    plt.ylabel('Média de Itens Perdidos')
    plt.xticks(rotation=45)
    plt.savefig('figuras/media_itens_perdidos_regiao.png')
    
    # Taxa de itens perdidos por região
    region_stats = orders_df.groupby('region').agg({
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    region_stats['missing_rate'] = region_stats['items_missing'] / region_stats['total_items'] * 100
    region_stats = region_stats.sort_values('missing_rate', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='region', y='missing_rate', data=region_stats)
    plt.title('Taxa de Itens Perdidos por Região (%)')
    plt.xlabel('Região')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.xticks(rotation=45)
    plt.savefig('figuras/taxa_itens_perdidos_regiao.png')
    
    results['region_stats'] = region_stats
    
    # ---------- 4. ANÁLISE TEMPORAL ----------
    print("\n4. Análise Temporal")
    
    # Tendência ao longo do tempo
    monthly_stats = orders_df.groupby('month').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    monthly_stats['missing_rate'] = monthly_stats['items_missing'] / monthly_stats['total_items'] * 100
    monthly_stats['month_name'] = monthly_stats['month'].apply(lambda x: calendar.month_name[x])
    
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.lineplot(x='month', y='order_id', data=monthly_stats, marker='o')
    plt.title('Número de Pedidos por Mês')
    plt.xlabel('Mês')
    plt.ylabel('Número de Pedidos')
    plt.xticks(monthly_stats['month'], rotation=45)
    
    plt.subplot(1, 2, 2)
    sns.lineplot(x='month', y='missing_rate', data=monthly_stats, marker='o')
    plt.title('Taxa de Itens Perdidos por Mês (%)')
    plt.xlabel('Mês')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.xticks(monthly_stats['month'], rotation=45)
    plt.tight_layout()
    plt.savefig('figuras/tendencia_temporal.png')
    
    # Análise por dia da semana
    day_stats = orders_df.groupby('day_of_week').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    day_stats['missing_rate'] = day_stats['items_missing'] / day_stats['total_items'] * 100
    day_stats['day_name'] = day_stats['day_of_week'].apply(lambda x: calendar.day_name[x])
    
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x='day_name', y='order_id', data=day_stats)
    plt.title('Número de Pedidos por Dia da Semana')
    plt.xlabel('Dia da Semana')
    plt.ylabel('Número de Pedidos')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    sns.barplot(x='day_name', y='missing_rate', data=day_stats)
    plt.title('Taxa de Itens Perdidos por Dia da Semana (%)')
    plt.xlabel('Dia da Semana')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figuras/analise_dia_semana.png')
    
    # Análise por hora do dia
    hour_stats = orders_df.groupby('delivery_hour_only').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    hour_stats['missing_rate'] = hour_stats['items_missing'] / hour_stats['total_items'] * 100
    
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x='delivery_hour_only', y='order_id', data=hour_stats)
    plt.title('Número de Pedidos por Hora do Dia')
    plt.xlabel('Hora do Dia')
    plt.ylabel('Número de Pedidos')
    
    plt.subplot(1, 2, 2)
    sns.barplot(x='delivery_hour_only', y='missing_rate', data=hour_stats)
    plt.title('Taxa de Itens Perdidos por Hora do Dia (%)')
    plt.xlabel('Hora do Dia')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.tight_layout()
    plt.savefig('figuras/analise_hora_dia.png')
    
    results['monthly_stats'] = monthly_stats
    results['day_stats'] = day_stats
    results['hour_stats'] = hour_stats
    
    # ---------- 5. SEGMENTAÇÃO ----------
    print("\n5. Segmentação e Análise de Grupos")
    
    # Análise por motorista
    print("\nAnalisando padrões por motorista...")
    driver_stats = orders_df.groupby('driver_id').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum',
        'order_amount': 'mean'
    }).reset_index()
    driver_stats['missing_rate'] = driver_stats['items_missing'] / driver_stats['total_items'] * 100
    driver_stats['orders_with_missing'] = orders_df[orders_df['items_missing'] > 0].groupby('driver_id').size().reset_index(name='orders_with_missing')['orders_with_missing']
    driver_stats['pct_orders_with_missing'] = (driver_stats['orders_with_missing'] / driver_stats['order_id']) * 100
    
    # Identificando motoristas com altas taxas de itens perdidos
    high_missing_drivers = driver_stats[driver_stats['order_id'] > 5].nlargest(10, 'missing_rate')
    
    plt.figure(figsize=(14, 6))
    sns.barplot(x='driver_id', y='missing_rate', data=high_missing_drivers)
    plt.title('Top 10 Motoristas com Maior Taxa de Itens Perdidos')
    plt.xlabel('ID do Motorista')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.xticks(rotation=90)
    plt.savefig('figuras/top_motoristas_taxa_perdidos.png')
    
    # Análise por cliente
    print("\nAnalisando padrões por cliente...")
    customer_stats = orders_df.groupby('customer_id').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum',
        'order_amount': 'mean'
    }).reset_index()
    customer_stats['missing_rate'] = customer_stats['items_missing'] / customer_stats['total_items'] * 100
    customer_stats['orders_with_missing'] = orders_df[orders_df['items_missing'] > 0].groupby('customer_id').size().reset_index(name='orders_with_missing')['orders_with_missing']
    customer_stats['pct_orders_with_missing'] = (customer_stats['orders_with_missing'] / customer_stats['order_id']) * 100
    
    # Identificando clientes com altas taxas de itens perdidos
    high_missing_customers = customer_stats[customer_stats['order_id'] > 3].nlargest(10, 'missing_rate')
    
    plt.figure(figsize=(14, 6))
    sns.barplot(x='customer_id', y='missing_rate', data=high_missing_customers)
    plt.title('Top 10 Clientes com Maior Taxa de Itens Perdidos')
    plt.xlabel('ID do Cliente')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.xticks(rotation=90)
    plt.savefig('figuras/top_clientes_taxa_perdidos.png')
    
    # Análise por categoria de produto
    print("\nAnalisando padrões por categoria de produto...")
    
    # Juntando dados de produtos perdidos com informações do produto
    missing_products = pd.merge(
        missing_items_df.melt(id_vars='order_id', value_vars=['product_id_1', 'product_id_2', 'product_id_3'], 
                             value_name='product_id').dropna(),
        products_df,
        on='product_id',
        how='left'
    )
    
    # Contagem de produtos perdidos por categoria
    missing_by_category = missing_products.groupby('category').size().reset_index(name='count')
    missing_by_category = missing_by_category.sort_values('count', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='category', y='count', data=missing_by_category)
    plt.title('Número de Produtos Perdidos por Categoria')
    plt.xlabel('Categoria')
    plt.ylabel('Número de Produtos Perdidos')
    plt.xticks(rotation=45)
    plt.savefig('figuras/produtos_perdidos_categoria.png')
    
    # Top 10 produtos mais reportados como perdidos
    missing_by_product = missing_products.groupby(['product_id', 'product_name', 'price']).size().reset_index(name='count')
    missing_by_product = missing_by_product.sort_values('count', ascending=False).head(10)
    
    plt.figure(figsize=(14, 6))
    sns.barplot(x='product_name', y='count', data=missing_by_product)
    plt.title('Top 10 Produtos Mais Reportados como Perdidos')
    plt.xlabel('Nome do Produto')
    plt.ylabel('Contagem')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('figuras/top_produtos_perdidos.png')
    
    # Análise por valor do produto
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='price', y='count', data=missing_by_product, s=100)
    for i, row in missing_by_product.iterrows():
        plt.annotate(row['product_name'], (row['price'], row['count']), fontsize=8)
    plt.title('Relação entre Preço do Produto e Frequência de Perda')
    plt.xlabel('Preço do Produto ($)')
    plt.ylabel('Frequência de Perda')
    plt.tight_layout()
    plt.savefig('figuras/relacao_preco_frequencia_perda.png')
    
    results['driver_stats'] = driver_stats
    results['customer_stats'] = customer_stats
    results['missing_by_category'] = missing_by_category
    results['missing_by_product'] = missing_by_product
    
    print("\nAnálise exploratória concluída! Visualizações foram salvas na pasta 'figuras/'")
    
    return results

def detect_fraud_patterns(conn, results, orders_df):
    """
    Detecta padrões de fraude nos dados
    """
    print("\n=== DETECÇÃO DE PADRÕES DE FRAUDE ===\n")
    
    driver_stats = results['driver_stats']
    customer_stats = results['customer_stats']
    
    # Detectando motoristas suspeitos usando Isolation Forest
    print("Detectando motoristas com comportamento anômalo...")
    
    # Selecionando apenas motoristas com número mínimo de entregas
    min_orders = 5
    drivers_for_analysis = driver_stats[driver_stats['order_id'] >= min_orders].copy()
    
    # Preparando os dados para detecção de anomalias
    X_drivers = drivers_for_analysis[['missing_rate', 'pct_orders_with_missing']].values
    
    # Aplicando Isolation Forest
    isolation_forest = IsolationForest(contamination=0.1, random_state=42)
    drivers_for_analysis['anomaly'] = isolation_forest.fit_predict(X_drivers)
    
    # Identificando motoristas suspeitos (anomalias são marcadas como -1)
    suspicious_drivers = drivers_for_analysis[drivers_for_analysis['anomaly'] == -1].sort_values('missing_rate', ascending=False)
    
    print(f"Identificados {len(suspicious_drivers)} motoristas com comportamento suspeito.")
    print("\nTop 5 motoristas mais suspeitos:")
    print(suspicious_drivers[['driver_id', 'order_id', 'missing_rate', 'pct_orders_with_missing']].head())
    
    # Visualizando os motoristas suspeitos
    plt.figure(figsize=(12, 8))
    plt.scatter(drivers_for_analysis[drivers_for_analysis['anomaly'] == 1]['missing_rate'], 
              drivers_for_analysis[drivers_for_analysis['anomaly'] == 1]['pct_orders_with_missing'],
              c='blue', label='Normal', alpha=0.7)
    plt.scatter(drivers_for_analysis[drivers_for_analysis['anomaly'] == -1]['missing_rate'], 
              drivers_for_analysis[drivers_for_analysis['anomaly'] == -1]['pct_orders_with_missing'],
              c='red', label='Suspeito', alpha=0.7)
    plt.xlabel('Taxa de Itens Perdidos (%)')
    plt.ylabel('% de Pedidos com Itens Perdidos')
    plt.title('Detecção de Motoristas Suspeitos')
    plt.legend()
    plt.grid(True)
    plt.savefig('figuras/motoristas_suspeitos.png')
    
    # Detectando clientes suspeitos usando Isolation Forest
    print("\nDetectando clientes com comportamento anômalo...")
    
    # Selecionando apenas clientes com número mínimo de pedidos
    min_orders = 3
    customers_for_analysis = customer_stats[customer_stats['order_id'] >= min_orders].copy()
    
    # Preparando os dados para detecção de anomalias
    X_customers = customers_for_analysis[['missing_rate', 'pct_orders_with_missing']].values
    
    # Aplicando Isolation Forest
    isolation_forest = IsolationForest(contamination=0.1, random_state=42)
    customers_for_analysis['anomaly'] = isolation_forest.fit_predict(X_customers)
    
    # Identificando clientes suspeitos (anomalias são marcadas como -1)
    suspicious_customers = customers_for_analysis[customers_for_analysis['anomaly'] == -1].sort_values('missing_rate', ascending=False)
    
    print(f"Identificados {len(suspicious_customers)} clientes com comportamento suspeito.")
    print("\nTop 5 clientes mais suspeitos:")
    print(suspicious_customers[['customer_id', 'order_id', 'missing_rate', 'pct_orders_with_missing']].head())
    
    # Visualizando os clientes suspeitos
    plt.figure(figsize=(12, 8))
    plt.scatter(customers_for_analysis[customers_for_analysis['anomaly'] == 1]['missing_rate'], 
              customers_for_analysis[customers_for_analysis['anomaly'] == 1]['pct_orders_with_missing'],
              c='blue', label='Normal', alpha=0.7)
    plt.scatter(customers_for_analysis[customers_for_analysis['anomaly'] == -1]['missing_rate'], 
              customers_for_analysis[customers_for_analysis['anomaly'] == -1]['pct_orders_with_missing'],
              c='red', label='Suspeito', alpha=0.7)
    plt.xlabel('Taxa de Itens Perdidos (%)')
    plt.ylabel('% de Pedidos com Itens Perdidos')
    plt.title('Detecção de Clientes Suspeitos')
    plt.legend()
    plt.grid(True)
    plt.savefig('figuras/clientes_suspeitos.png')
    
    # Analisando padrões por valor do pedido
    print("\nAnalisando relação entre valor do pedido e itens perdidos...")
    
    # Agrupando pedidos por faixas de valor
    orders_df['value_range'] = pd.cut(orders_df['order_amount'], 
                                     bins=[0, 100, 300, 500, 1000, 10000],
                                     labels=['0-100', '100-300', '300-500', '500-1000', '1000+'])
    
    value_range_stats = orders_df.groupby('value_range').agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    
    value_range_stats['missing_rate'] = value_range_stats['items_missing'] / value_range_stats['total_items'] * 100
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='value_range', y='missing_rate', data=value_range_stats)
    plt.title('Taxa de Itens Perdidos por Faixa de Valor do Pedido')
    plt.xlabel('Faixa de Valor ($)')
    plt.ylabel('Taxa de Itens Perdidos (%)')
    plt.savefig('figuras/taxa_perdidos_por_valor.png')
    
    # Identificando padrões suspeitos específicos
    print("\nIdentificando padrões suspeitos específicos...")
    
    # Padrão 1: Motoristas que têm alta taxa de perda em horários específicos
    driver_hour_stats = orders_df.groupby(['driver_id', 'delivery_hour_only']).agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    
    driver_hour_stats['missing_rate'] = driver_hour_stats['items_missing'] / driver_hour_stats['total_items'] * 100
    driver_hour_high_missing = driver_hour_stats[(driver_hour_stats['order_id'] >= 3) & 
                                               (driver_hour_stats['missing_rate'] >= 30)].sort_values('missing_rate', ascending=False)
    
    print("\nMotoristas com alta taxa de perda em horários específicos:")
    print(driver_hour_high_missing[['driver_id', 'delivery_hour_only', 'order_id', 'missing_rate']].head(5))
    
    # Padrão 2: Motoristas com alta taxa de perda em regiões específicas
    driver_region_stats = orders_df.groupby(['driver_id', 'region']).agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    
    driver_region_stats['missing_rate'] = driver_region_stats['items_missing'] / driver_region_stats['total_items'] * 100
    driver_region_high_missing = driver_region_stats[(driver_region_stats['order_id'] >= 3) & 
                                                  (driver_region_stats['missing_rate'] >= 30)].sort_values('missing_rate', ascending=False)
    
    print("\nMotoristas com alta taxa de perda em regiões específicas:")
    print(driver_region_high_missing[['driver_id', 'region', 'order_id', 'missing_rate']].head(5))
    
    # Padrão 3: Clientes que reportam itens perdidos em dias específicos
    customer_day_stats = orders_df.groupby(['customer_id', 'day_name']).agg({
        'order_id': 'count',
        'items_missing': 'sum',
        'total_items': 'sum'
    }).reset_index()
    
    customer_day_stats['missing_rate'] = customer_day_stats['items_missing'] / customer_day_stats['total_items'] * 100
    customer_day_high_missing = customer_day_stats[(customer_day_stats['order_id'] >= 3) & 
                                                (customer_day_stats['missing_rate'] >= 30)].sort_values('missing_rate', ascending=False)
    
    print("\nClientes que reportam itens perdidos em dias específicos:")
    print(customer_day_high_missing[['customer_id', 'day_name', 'order_id', 'missing_rate']].head(5))
    
    suspicious_patterns = {
        'driver_hour_high_missing': driver_hour_high_missing,
        'driver_region_high_missing': driver_region_high_missing,
        'customer_day_high_missing': customer_day_high_missing
    }
    
    print("\nDetecção de padrões de fraude concluída!")
    
    return suspicious_drivers, suspicious_customers, suspicious_patterns

def main():
    """
    Função principal que executa todo o pipeline de análise
    """
    print("=== PROJETO DE DETECÇÃO DE FRAUDES EM ENTREGAS DO WALMART ===\n")
    
    # Carregar os dados
    conn, orders_df, drivers_df, customers_df, missing_items_df, products_df = load_data()
    
    # Realizar análise exploratória dos dados
    results = exploratory_data_analysis(conn, orders_df, drivers_df, customers_df, missing_items_df, products_df)
    
    # Detectar padrões de fraude
    suspicious_drivers, suspicious_customers, suspicious_patterns = detect_fraud_patterns(conn, results, orders_df)
    
    # Fechar a conexão com o banco de dados
    conn.close()
    
    print("\n=== RESUMO DAS DESCOBERTAS ===")
    
    # Resumo das análises por região
    region_stats = results['region_stats'].sort_values('missing_rate', ascending=False)
    print("\nRegiões com maior taxa de itens perdidos:")
    for i, row in region_stats.head(3).iterrows():
        print(f"• {row['region']}: {row['missing_rate']:.2f}% de itens reportados como não entregues")
    
    # Resumo dos motoristas suspeitos
    print("\nMotoristas com comportamento suspeito:")
    for i, driver in enumerate(suspicious_drivers.head(5).iterrows()):
        driver_data = driver[1]
        print(f"• ID: {driver_data['driver_id']}, Nome: {driver_data['driver_name']}")
        print(f"  {driver_data['missing_items_count']} itens reportados como não entregues em {driver_data['orders_with_missing']} pedidos")
        print(f"  Taxa de problemas: {driver_data['problem_rate']:.2f}%")
    
    # Resumo dos clientes suspeitos
    print("\nClientes com comportamento suspeito:")
    for i, customer in enumerate(suspicious_customers.head(5).iterrows()):
        customer_data = customer[1]
        print(f"• ID: {customer_data['customer_id']}, Nome: {customer_data['customer_name']}")
        print(f"  {customer_data['missing_items_count']} itens reportados como não entregues em {customer_data['orders_with_missing']} pedidos")
        print(f"  Taxa de problemas: {customer_data['problem_rate']:.2f}%")
    
    # Padrões de produtos mais frequentemente reportados como não entregues
    print("\nCategorias de produtos mais reportados como não entregues:")
    for i, product in enumerate(suspicious_patterns['product_categories'].head(3).iterrows()):
        product_data = product[1]
        print(f"• {product_data['category']}: {product_data['missing_count']} ocorrências, {product_data['missing_rate']:.2f}% de taxa de não entrega")
    
    # Horários com maior incidência de problemas
    print("\nHorários com maior incidência de problemas:")
    for i, time_slot in enumerate(suspicious_patterns['time_slots'].head(3).iterrows()):
        time_data = time_slot[1]
        print(f"• {time_data['time_slot']}: {time_data['missing_rate']:.2f}% de taxa de não entrega")
    
    # Criar e treinar modelo preditivo para detecção de fraudes
    print("\n=== CRIANDO MODELO PREDITIVO ===")
    model, train_features, test_features, train_labels, test_labels = train_fraud_detection_model(results, orders_df, drivers_df, customers_df)
    
    # Avaliar o modelo
    accuracy, precision, recall, f1, auc = evaluate_model(model, test_features, test_labels)
    
    print(f"\nDesempenho do modelo:")
    print(f"• Acurácia: {accuracy:.2f}")
    print(f"• Precisão: {precision:.2f}")
    print(f"• Recall: {recall:.2f}")
    print(f"• F1-Score: {f1:.2f}")
    print(f"• AUC-ROC: {auc:.2f}")
    
    # Identificar características mais importantes para predição
    features_importance = get_feature_importance(model, train_features.columns)
    print("\nCaracterísticas mais importantes para detecção de fraudes:")
    for feature, importance in features_importance[:5]:
        print(f"• {feature}: {importance:.2f}")
    
    # Gerar recomendações baseadas na análise
    print("\n=== RECOMENDAÇÕES ===")
    recommendations = generate_recommendations(results, suspicious_drivers, suspicious_customers, suspicious_patterns)
    
    for i, recommendation in enumerate(recommendations[:5], 1):
        print(f"{i}. {recommendation}")
    
    # Salvar resultados e gerar visualizações
    print("\n=== SALVANDO RESULTADOS E GERANDO VISUALIZAÇÕES ===")
    save_results(results, suspicious_drivers, suspicious_customers, 
                suspicious_patterns, model, accuracy, precision, recall, f1, auc)
    
    generate_dashboard(results, suspicious_drivers, suspicious_customers, suspicious_patterns)
    
    print("\n=== ANÁLISE CONCLUÍDA ===")
    print("O relatório completo, dashboard e recomendações estão disponíveis nos arquivos gerados")
    
    # Estimativa de redução de fraudes
    current_fraud_loss = 400000000  # 400 milhões de dólares (aumento de 2022 para 2023)
    estimated_reduction = calculate_fraud_reduction_estimate(results, model)
    estimated_savings = current_fraud_loss * estimated_reduction / 100
    
    print(f"\nEstimativa de redução de fraudes com as medidas sugeridas: {estimated_reduction:.1f}%")
    print(f"Economia potencial anual: ${estimated_savings/1000000:.1f} milhões")


def load_data():
    """
    Carrega os dados do banco SQLite e retorna os DataFrames
    """
    print("Carregando dados...")
    
    # Caminho para o banco de dados SQLite
    db_path = r'C:\Users\louis\datatech\Database\walmart_fraudes.db'
    
    try:
        # Conectando ao banco
        conn = sqlite3.connect(db_path)
        
        # Carregando os dados em DataFrames
        orders_df = pd.read_sql("SELECT * FROM orders", conn)
        drivers_df = pd.read_sql("SELECT * FROM drivers", conn)
        customers_df = pd.read_sql("SELECT * FROM customers", conn)
        missing_items_df = pd.read_sql("SELECT * FROM missing_items", conn)
        products_df = pd.read_sql("SELECT * FROM products", conn)
        
        # Formatando os dados
        orders_df['date'] = pd.to_datetime(orders_df['date'])
        
        # Verificação básica da carga de dados
        print(f"Dados carregados com sucesso:")
        print(f"• Pedidos: {len(orders_df)} registros")
        print(f"• Motoristas: {len(drivers_df)} registros")
        print(f"• Clientes: {len(customers_df)} registros")
        print(f"• Produtos não entregues: {len(missing_items_df)} registros")
        print(f"• Produtos: {len(products_df)} registros")
        
        return conn, orders_df, drivers_df, customers_df, missing_items_df, products_df
        
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        exit(1)


def exploratory_data_analysis(conn, orders_df, drivers_df, customers_df, missing_items_df, products_df):
    """
    Realiza análise exploratória dos dados e retorna os resultados
    """
    print("\nRealizando análise exploratória dos dados...")
    results = {}
    
    # Estatísticas gerais
    total_orders = len(orders_df)
    total_items_delivered = orders_df['items_delivered'].sum()
    total_items_missing = orders_df['items_missing'].sum()
    missing_rate = (total_items_missing / (total_items_delivered + total_items_missing)) * 100
    
    orders_with_missing = orders_df[orders_df['items_missing'] > 0].shape[0]
    orders_with_missing_rate = (orders_with_missing / total_orders) * 100
    
    print(f"• Total de pedidos: {total_orders}")
    print(f"• Pedidos com itens faltantes: {orders_with_missing} ({orders_with_missing_rate:.2f}%)")
    print(f"• Taxa média de itens não entregues: {missing_rate:.2f}%")
    
    results['general_stats'] = {
        'total_orders': total_orders,
        'total_items_delivered': total_items_delivered,
        'total_items_missing': total_items_missing,
        'missing_rate': missing_rate,
        'orders_with_missing': orders_with_missing,
        'orders_with_missing_rate': orders_with_missing_rate
    }
    
    # Análise por região
    region_stats = orders_df.groupby('region').agg(
        total_orders=('order_id', 'count'),
        items_delivered=('items_delivered', 'sum'),
        items_missing=('items_missing', 'sum'),
        orders_with_missing=('items_missing', lambda x: (x > 0).sum())
    ).reset_index()
    
    region_stats['missing_rate'] = (region_stats['items_missing'] / 
                                   (region_stats['items_delivered'] + region_stats['items_missing'])) * 100
    region_stats['problem_order_rate'] = (region_stats['orders_with_missing'] / region_stats['total_orders']) * 100
    
    results['region_stats'] = region_stats
    
    # Análise por motorista
    driver_stats = pd.read_sql("""
        SELECT 
            d.driver_id,
            d.driver_name,
            d.age,
            COUNT(o.order_id) AS orders_delivered,
            SUM(o.items_missing) AS missing_items_count,
            SUM(o.items_delivered) AS delivered_items_count,
            SUM(CASE WHEN o.items_missing > 0 THEN 1 ELSE 0 END) AS orders_with_missing
        FROM 
            drivers d
        JOIN 
            orders o ON d.driver_id = o.driver_id
        GROUP BY 
            d.driver_id
    """, conn)
    
    driver_stats['missing_rate'] = (driver_stats['missing_items_count'] / 
                                   (driver_stats['delivered_items_count'] + driver_stats['missing_items_count'])) * 100
    driver_stats['problem_rate'] = (driver_stats['orders_with_missing'] / driver_stats['orders_delivered']) * 100
    
    results['driver_stats'] = driver_stats
    
    # Análise por cliente
    customer_stats = pd.read_sql("""
        SELECT 
            c.customer_id,
            c.customer_name,
            c.customer_age,
            COUNT(o.order_id) AS orders_placed,
            SUM(o.items_missing) AS missing_items_count,
            SUM(o.items_delivered) AS delivered_items_count,
            SUM(CASE WHEN o.items_missing > 0 THEN 1 ELSE 0 END) AS orders_with_missing
        FROM 
            customers c
        JOIN 
            orders o ON c.customer_id = o.customer_id
        GROUP BY 
            c.customer_id
    """, conn)
    
    customer_stats['missing_rate'] = (customer_stats['missing_items_count'] / 
                                     (customer_stats['delivered_items_count'] + customer_stats['missing_items_count'])) * 100
    customer_stats['problem_rate'] = (customer_stats['orders_with_missing'] / customer_stats['orders_placed']) * 100
    
    results['customer_stats'] = customer_stats
    
    # Análise por hora do dia
    orders_df['hour_of_day'] = orders_df['delivery_hour_only']
    
    time_stats = orders_df.groupby('hour_of_day').agg(
        total_orders=('order_id', 'count'),
        items_delivered=('items_delivered', 'sum'),
        items_missing=('items_missing', 'sum'),
        orders_with_missing=('items_missing', lambda x: (x > 0).sum())
    ).reset_index()
    
    time_stats['missing_rate'] = (time_stats['items_missing'] / 
                                 (time_stats['items_delivered'] + time_stats['items_missing'])) * 100
    time_stats['problem_order_rate'] = (time_stats['orders_with_missing'] / time_stats['total_orders']) * 100
    
    results['time_stats'] = time_stats
    
    # Análise de produtos mais frequentemente reportados como não entregues
    # Unindo missing_items com products para obter categorias e preços
    missing_products_stats = pd.DataFrame()
    
    for col in ['product_id_1', 'product_id_2', 'product_id_3']:
        if col in missing_items_df.columns:
            temp_df = missing_items_df[missing_items_df[col].notnull()]
            temp_df = temp_df.rename(columns={col: 'product_id'})
            temp_df = temp_df[['product_id']]
            if missing_products_stats.empty:
                missing_products_stats = temp_df
            else:
                missing_products_stats = pd.concat([missing_products_stats, temp_df])
    
    missing_products_stats = missing_products_stats.merge(
        products_df[['product_id', 'product_name', 'category', 'price']], 
        on='product_id', 
        how='left'
    )
    
    product_category_stats = missing_products_stats.groupby('category').agg(
        missing_count=('product_id', 'count'),
        avg_price=('price', 'mean')
    ).reset_index()
    
    # Calcular a taxa de não entrega por categoria
    # Precisamos do total de produtos de cada categoria entregues
    # Como aproximação, vamos usar a proporção de produtos na tabela de produtos
    category_counts = products_df['category'].value_counts().reset_index()
    category_counts.columns = ['category', 'total_products']
    
    product_category_stats = product_category_stats.merge(category_counts, on='category', how='left')
    product_category_stats['missing_rate'] = (product_category_stats['missing_count'] / 
                                             product_category_stats['total_products']) * 100
    
    results['product_category_stats'] = product_category_stats
    
    # Análise por valor do pedido
    orders_df['order_amount_range'] = pd.cut(
        orders_df['order_amount'], 
        bins=[0, 100, 250, 500, 1000, float('inf')],
        labels=['$0-$100', '$100-$250', '$250-$500', '$500-$1000', '$1000+']
    )
    
    amount_stats = orders_df.groupby('order_amount_range').agg(
        total_orders=('order_id', 'count'),
        items_delivered=('items_delivered', 'sum'),
        items_missing=('items_missing', 'sum'),
        orders_with_missing=('items_missing', lambda x: (x > 0).sum())
    ).reset_index()
    
    amount_stats['missing_rate'] = (amount_stats['items_missing'] / 
                                   (amount_stats['items_delivered'] + amount_stats['items_missing'])) * 100
    amount_stats['problem_order_rate'] = (amount_stats['orders_with_missing'] / amount_stats['total_orders']) * 100
    
    results['amount_stats'] = amount_stats
    
    # Análise por período do dia
    period_stats = orders_df.groupby('period_of_day').agg(
        total_orders=('order_id', 'count'),
        items_delivered=('items_delivered', 'sum'),
        items_missing=('items_missing', 'sum'),
        orders_with_missing=('items_missing', lambda x: (x > 0).sum())
    ).reset_index()
    
    period_stats['missing_rate'] = (period_stats['items_missing'] / 
                                  (period_stats['items_delivered'] + period_stats['items_missing'])) * 100
    period_stats['problem_order_rate'] = (period_stats['orders_with_missing'] / period_stats['total_orders']) * 100
    
    results['period_stats'] = period_stats
    
    print("Análise exploratória concluída com sucesso!")
    return results


def detect_fraud_patterns(conn, results, orders_df):
    """
    Detecta padrões de fraude nos dados analisados
    """
    print("\nDetectando padrões de fraude...")
    
    # Identificar motoristas suspeitos
    driver_stats = results['driver_stats']
    overall_missing_rate = results['general_stats']['missing_rate']
    overall_problem_rate = results['general_stats']['orders_with_missing_rate']
    
    # Definindo limiares para detecção de motoristas suspeitos
    # Motoristas com taxa de itens faltantes muito acima da média
    missing_rate_threshold = overall_missing_rate * 1.5
    problem_rate_threshold = overall_problem_rate * 1.5
    
    suspicious_drivers = driver_stats[
        (driver_stats['missing_rate'] > missing_rate_threshold) &
        (driver_stats['problem_rate'] > problem_rate_threshold) &
        (driver_stats['orders_delivered'] > 5)  # Pelo menos 5 entregas para ser estatisticamente relevante
    ].sort_values('problem_rate', ascending=False)
    
    print(f"• Identificados {len(suspicious_drivers)} motoristas com comportamento suspeito")
    
    # Identificar clientes suspeitos
    customer_stats = results['customer_stats']
    
    suspicious_customers = customer_stats[
        (customer_stats['missing_rate'] > missing_rate_threshold) &
        (customer_stats['problem_rate'] > problem_rate_threshold) &
        (customer_stats['orders_placed'] > 5)  # Pelo menos 5 pedidos para ser estatisticamente relevante
    ].sort_values('problem_rate', ascending=False)
    
    print(f"• Identificados {len(suspicious_customers)} clientes com comportamento suspeito")
    
    # Identificar padrões de tempo/região/produto
    suspicious_patterns = {}
    
    # Horários mais problemáticos
    time_slots = results['time_stats'].copy()
    time_slots.columns = ['time_slot' if col == 'hour_of_day' else col for col in time_slots.columns]
    suspicious_patterns['time_slots'] = time_slots[
        (time_slots['missing_rate'] > overall_missing_rate * 1.2) &
        (time_slots['total_orders'] > 50)  # Número mínimo de pedidos para ser estatisticamente relevante
    ].sort_values('missing_rate', ascending=False)
    
    # Categorias de produtos mais frequentemente relatadas como não entregues
    suspicious_patterns['product_categories'] = results['product_category_stats'].sort_values('missing_rate', ascending=False)
    
    # Regiões com maior incidência de problemas
    suspicious_patterns['regions'] = results['region_stats'][
        (results['region_stats']['missing_rate'] > overall_missing_rate * 1.2) &
        (results['region_stats']['total_orders'] > 50)  # Número mínimo de pedidos para ser estatisticamente relevante
    ].sort_values('missing_rate', ascending=False)
    
    # Padrões de valor de pedidos
    suspicious_patterns['order_amounts'] = results['amount_stats'][
        results['amount_stats']['missing_rate'] > overall_missing_rate * 1.2
    ].sort_values('missing_rate', ascending=False)
    
    # Verificar combinações específicas (motorista-região, motorista-horário, etc.)
    driver_region_stats = pd.read_sql("""
        SELECT 
            d.driver_id,
            o.region,
            COUNT(o.order_id) AS total_orders,
            SUM(o.items_missing) AS items_missing,
            SUM(o.items_delivered) AS items_delivered,
            SUM(CASE WHEN o.items_missing > 0 THEN 1 ELSE 0 END) AS orders_with_missing
        FROM 
            drivers d
        JOIN 
            orders o ON d.driver_id = o.driver_id
        GROUP BY 
            d.driver_id, o.region
        HAVING 
            total_orders > 3
    """, conn)
    
    driver_region_stats['missing_rate'] = (driver_region_stats['items_missing'] / 
                                         (driver_region_stats['items_delivered'] + driver_region_stats['items_missing'])) * 100
    driver_region_stats['problem_rate'] = (driver_region_stats['orders_with_missing'] / driver_region_stats['total_orders']) * 100
    
    suspicious_patterns['driver_region'] = driver_region_stats[
        (driver_region_stats['missing_rate'] > missing_rate_threshold) &
        (driver_region_stats['problem_rate'] > problem_rate_threshold)
    ].sort_values('problem_rate', ascending=False)
    
    # Encontrar padrões de produtos específicos por motorista
    # Esta análise exigiria unir várias tabelas e ficaria demasiado complexa neste código de exemplo
    
    print("Detecção de padrões de fraude concluída!")
    return suspicious_drivers, suspicious_customers, suspicious_patterns


def train_fraud_detection_model(results, orders_df, drivers_df, customers_df):
    """
    Treina um modelo de machine learning para detectar possíveis fraudes
    """
    print("Treinando modelo de detecção de fraudes...")
    
    # Preparar conjunto de dados para treino
    # Unir dados de pedidos com estatísticas dos motoristas e clientes
    model_data = orders_df.copy()
    
    # Adicionar recursos do motorista
    driver_features = results['driver_stats'][['driver_id', 'age', 'missing_rate', 'problem_rate']].rename(
        columns={'missing_rate': 'driver_missing_rate', 'problem_rate': 'driver_problem_rate'}
    )
    model_data = model_data.merge(driver_features, on='driver_id', how='left')
    
    # Adicionar recursos do cliente
    customer_features = results['customer_stats'][['customer_id', 'customer_age', 'missing_rate', 'problem_rate']].rename(
        columns={'missing_rate': 'customer_missing_rate', 'problem_rate': 'customer_problem_rate'}
    )
    model_data = model_data.merge(customer_features, on='customer_id', how='left')
    
    # Adicionar recursos da região
    region_features = results['region_stats'][['region', 'missing_rate', 'problem_order_rate']].rename(
        columns={'missing_rate': 'region_missing_rate', 'problem_order_rate': 'region_problem_rate'}
    )
    model_data = model_data.merge(region_features, on='region', how='left')
    
    # Criar variável alvo
    # Definimos como "suspeito" um pedido onde:
    # - Mais de 20% dos itens foram relatados como não entregues
    # - O motorista tem alta taxa de problemas
    # - O cliente tem alta taxa de problemas 
    # - A taxa de itens não entregues é maior que a média da região
    
    model_data['missing_rate'] = (model_data['items_missing'] / 
                                 (model_data['items_delivered'] + model_data['items_missing'])) * 100
    
    # Definir limiar para classificar um pedido como suspeito
    overall_missing_rate = results['general_stats']['missing_rate']
    
    model_data['suspicious'] = (
        (model_data['missing_rate'] > 20) | 
        ((model_data['driver_problem_rate'] > results['general_stats']['orders_with_missing_rate'] * 1.5) & 
         (model_data['missing_rate'] > overall_missing_rate)) |
        ((model_data['customer_problem_rate'] > results['general_stats']['orders_with_missing_rate'] * 1.5) & 
         (model_data['missing_rate'] > overall_missing_rate)) |
        ((model_data['missing_rate'] > model_data['region_missing_rate'] * 1.5) & 
         (model_data['missing_rate'] > 10))
    ).astype(int)
    
    # Criar recursos (features) para o modelo
    # Transformar variáveis categóricas
    model_data['hour'] = model_data['delivery_hour_only']
    model_data['month'] = pd.to_datetime(model_data['date']).dt.month
    model_data['day_of_week'] = pd.to_datetime(model_data['date']).dt.dayofweek
    
    # Codificar variáveis categóricas
    model_data = pd.get_dummies(model_data, columns=['region', 'period_of_day'], drop_first=True)
    
    # Selecionar recursos para o modelo
    features = [
        'order_amount', 'items_delivered', 'items_missing', 'hour', 
        'month', 'day_of_week', 'age', 'driver_missing_rate', 'driver_problem_rate',
        'customer_age', 'customer_missing_rate', 'customer_problem_rate', 
        'region_missing_rate', 'region_problem_rate'
    ] + [col for col in model_data.columns if col.startswith('region_') or col.startswith('period_of_day_')]
    
    # Remover linhas com valores ausentes
    model_data = model_data.dropna(subset=features + ['suspicious'])
    
    # Dividir os dados em treino e teste
    from sklearn.model_selection import train_test_split
    
    X = model_data[features]
    y = model_data['suspicious']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Treinar modelo
    from sklearn.ensemble import RandomForestClassifier
    
    model = RandomForestClassifier(
        n_estimators=100, 
        max_depth=10,
        min_samples_split=10,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    print(f"Modelo treinado com {len(X_train)} amostras.")
    return model, X_train, X_test, y_train, y_test


def evaluate_model(model, X_test, y_test):
    """
    Avalia o desempenho do modelo de detecção de fraudes
    """
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    
    # Fazer previsões
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    return accuracy, precision, recall, f1, auc


def get_feature_importance(model, feature_names):
    """
    Retorna a importância das características no modelo
    """
    importances = model.feature_importances_
    feature_importance = list(zip(feature_names, importances))
    return sorted(feature_importance, key=lambda x: x[1], reverse=True)


def generate_recommendations(results, suspicious_drivers, suspicious_customers, suspicious_patterns):
    """
    Gera recomendações baseadas na análise dos dados
    """
    recommendations = [
        "Implementar um sistema de verificação fotográfica para entregas de alto valor.",
        "Realizar auditorias periódicas nos motoristas com alta taxa de itens relatados como não entregues.",
        "Implementar um sistema de assinatura digital para confirmação de entrega completa.",
        "Estabelecer um programa de treinamento e conscientização para motoristas sobre a importância da entrega completa.",
        "Implementar um sistema de geolocalização para verificar se o motorista esteve no endereço de entrega pelo tempo adequado.",
        "Criar um sistema de avaliação cruzada para clientes que reportam itens não entregues com frequência.",
        "Implementar um sistema de alertas para pedidos com produtos de categorias frequentemente relatadas como não entregues.",
        "Estabelecer um programa de incentivos para motoristas com baixas taxas de problemas relatados.",
        "Desenvolver um algoritmo de priorização para investigações de casos de não entrega baseado no valor do item e no histórico do motorista e cliente.",
        "Implementar embalagens especiais com selo de segurança para produtos de maior valor.",
        "Realizar pesquisas de satisfação direcionadas após cada entrega para avaliar a qualidade do serviço.",
        "Atribuir motoristas mais experientes para entregas em regiões com alta taxa de problemas."
    ]
    
    return recommendations


def save_results(results, suspicious_drivers, suspicious_customers, suspicious_patterns, model, accuracy, precision, recall, f1, auc):
    """
    Salva os resultados da análise em arquivos para referência futura
    """
    import os
    
    # Criar diretório para resultados se não existir
    os.makedirs("resultados", exist_ok=True)
    
    # Salvar estatísticas gerais
    with open("resultados/estatisticas_gerais.txt", "w") as f:
        f.write("ESTATÍSTICAS GERAIS\n")
        f.write("===================\n\n")
        f.write(f"Total de pedidos: {results['general_stats']['total_orders']}\n")
        f.write(f"Total de itens entregues: {results['general_stats']['total_items_delivered']}\n")
        f.write(f"Total de itens não entregues: {results['general_stats']['total_items_not_delivered']}\n")
        f.write(f"Taxa de não entrega: {results['general_stats']['non_delivery_rate']:.2f}%\n")
        f.write(f"Valor total de pedidos: R$ {results['general_stats']['total_order_value']:.2f}\n")
        f.write(f"Valor médio por pedido: R$ {results['general_stats']['avg_order_value']:.2f}\n\n")
    
    # Salvar motoristas suspeitos
    with open("resultados/motoristas_suspeitos.csv", "w") as f:
        f.write("id_motorista,nome,total_pedidos,taxa_nao_entrega,score_suspeita\n")
        for driver in suspicious_drivers:
            f.write(f"{driver['id']},{driver['name']},{driver['total_orders']},{driver['non_delivery_rate']:.2f},{driver['suspicion_score']:.2f}\n")
    
    # Salvar clientes suspeitos
    with open("resultados/clientes_suspeitos.csv", "w") as f:
        f.write("id_cliente,nome,total_pedidos,taxa_reclamacao,score_suspeita\n")
        for customer in suspicious_customers:
            f.write(f"{customer['id']},{customer['name']},{customer['total_orders']},{customer['complaint_rate']:.2f},{customer['suspicion_score']:.2f}\n")
    
    # Salvar padrões suspeitos
    with open("resultados/padroes_suspeitos.txt", "w") as f:
        f.write("PADRÕES SUSPEITOS IDENTIFICADOS\n")
        f.write("==============================\n\n")
        for i, pattern in enumerate(suspicious_patterns, 1):
            f.write(f"Padrão {i}:\n")
            f.write(f"Descrição: {pattern['description']}\n")
            f.write(f"Ocorrências: {pattern['occurrences']}\n")
            f.write(f"Confiança: {pattern['confidence']:.2f}\n")
            f.write(f"Impacto estimado: R$ {pattern['estimated_impact']:.2f}\n\n")
    
    # Salvar métricas do modelo
    with open("resultados/metricas_modelo.txt", "w") as f:
        f.write("MÉTRICAS DO MODELO DE DETECÇÃO\n")
        f.write("=============================\n\n")
        f.write(f"Modelo utilizado: {model}\n")
        f.write(f"Acurácia: {accuracy:.4f}\n")
        f.write(f"Precisão: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1-Score: {f1:.4f}\n")
        f.write(f"AUC-ROC: {auc:.4f}\n")
    
    # Opcional: exportar dados completos em formato JSON para análise futura
    import json
    with open("resultados/dados_completos.json", "w") as f:
        json.dump(results, f, indent=4)
    
    print("Resultados salvos com sucesso no diretório 'resultados/'")